## Implementation of the language models

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai.text.models import * 

[`text.models`](/text.models.html#text.models) module fully implements the encoder for an [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf), the [transformer model](https://arxiv.org/abs/1706.03762) and the [transformer XL model](https://arxiv.org/abs/1901.02860). They can then be plugged in with a decoder to make a language model, or some classifying layers to make a text classifier.

## Language model modules

In [ ]:
show_doc(AWD_LSTM, title_level=3)

<h3 id="AWD_LSTM" class="doc_header"><code>class</code> <code>AWD_LSTM</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L75" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AWD_LSTM-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AWD_LSTM</code>(**`vocab_sz`**:`int`, **`emb_sz`**:`int`, **`n_hid`**:`int`, **`n_layers`**:`int`, **`pad_token`**:`int`=***`1`***, **`hidden_p`**:`float`=***`0.2`***, **`input_p`**:`float`=***`0.6`***, **`embed_p`**:`float`=***`0.1`***, **`weight_p`**:`float`=***`0.5`***, **`qrnn`**:`bool`=***`False`***, **`bidir`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="AWD_LSTM-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AWD_LSTM-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AWD_LSTM</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182.  

The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a [`RNNDropout`](/text.models.awd_lstm.html#RNNDropout) module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are a total four different dropouts in the encoder of the AWD-LSTM:

- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector. This is the `embed_p` parameter.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension). This is the `input_p` parameter.
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated. This is the `weight_p` parameter.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). It isn’t applied to the last output (which will get its own dropout in the decoder).This is the `hidden_p` parameter.

The other attributes are `vocab_sz` for the number of tokens in your vocabulary, `emb_sz` for the embedding size, `n_hid` for the hidden size of your inner LSTMs (or QRNNs), `n_layers` the number of layers and `pad_token` for the index of an eventual padding token (1 by default in fastai).

The flag `qrnn=True` replace the inner LSTMs by [QRNNs](https://arxiv.org/abs/1611.01576).

In [ ]:
show_doc(AWD_LSTM.reset)

<h4 id="AWD_LSTM.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L132" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AWD_LSTM-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="AWD_LSTM-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AWD_LSTM-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reset the hidden states.  

In [ ]:
show_doc(Transformer, title_level=3)

<h3 id="Transformer" class="doc_header"><code>class</code> <code>Transformer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L149" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#Transformer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>Transformer</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***, **`learned_pos_enc`**:`bool`=***`True`***, **`mask`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="Transformer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#Transformer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>Transformer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Transformer model: https://arxiv.org/abs/1706.03762.  

The main idea of this article is to use regular neural net for NLP instead of an RNN, but with lots of attention layers. Intuitively, those attention layers tell the model to pay more interest to this or that word when trying to predict its output. 

It starts from embeddings from `vocab_sz` (number of tokens) to `d_model` (which is basically the hidden size throughout the model), and it will look at inputs of size batch_size by `ctx_len` (for context length). We add a positional encoding to the embeddings (since a regular neural net has no idea of the order of words), either learned or coming from [`PositionalEncoding`](/text.models.transformer.html#PositionalEncoding) depending on `learned_pos_enc`. We then have a dropout of `embed_p` followed by `n_layers` blocks of [`MultiHeadAttention`](/text.models.transformer.html#MultiHeadAttention) followed by [`feed_forward`](/text.models.transformer.html#feed_forward). 

In the attention we use `n_heads` with each a hidden state of `d_head` (will default to `d_model//n_heads`). If `mask=True`, a mask will make sure no attention is paid to future tokens (which would be cheating when training a language model). If `scale=True`, the attention scores are scaled by a factor `1 / math.sqrt(d_head)`. A dropout of `attn_p` is applied to the attention scores, then the final result get applied a dropout of `resid_p` before being summed to the original input (residual connection before the layer norm).

In feed forward, we have two linear layers from `d_model` to `d_inner` and then back. Those have `bias` if that flag is `True` and a dropout of `ff_p` is applied, after each if `double_drop=True`, or just at the end otherwise. `act` is used in the middle as a non-linearity. 

In [ ]:
show_doc(TransformerXL, title_level=3)

<h3 id="TransformerXL" class="doc_header"><code>class</code> <code>TransformerXL</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L175" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#TransformerXL-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>TransformerXL</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`False`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadRelativeAttention'`***, **`learned_pos_enc`**:`bool`=***`False`***, **`mask`**:`bool`=***`True`***, **`mem_len`**:`int`=***`0`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="TransformerXL-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#TransformerXL-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>TransformerXL</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

TransformerXL model: https://arxiv.org/abs/1901.02860.  

TransformerXL is a transformer architecture with a sort of hidden state formed by the results of the intermediate layers on previous tokens. Its size is determined by `mem_len`. By using this context, those models are capable of learning longer dependencies and can also be used for faster text generation at inference: a regular transformer model would have to reexamine the whole of sequence of indexes generated so far, whereas we can feed the new tokens one by one to a transformer XL (like we do with a regular RNN).

In [ ]:
show_doc(TransformerXL.reset)

<h4 id="TransformerXL.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L192" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#TransformerXL-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="TransformerXL-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#TransformerXL-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reset the internal memory.  

## Decoders

In [ ]:
show_doc(LinearDecoder, title_level=3)

<h3 id="LinearDecoder" class="doc_header"><code>class</code> <code>LinearDecoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L138" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LinearDecoder-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>LinearDecoder</code>(**`n_out`**:`int`, **`n_hid`**:`int`, **`output_p`**:`float`, **`tie_encoder`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`bias`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="LinearDecoder-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LinearDecoder-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>LinearDecoder</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

To go on top of a RNNCore module and create a Language Model.  

Create a the decoder to go on top of an [`RNNCore`](/text.models.awd_lstm.html#RNNCore) encoder and create a language model. `n_hid` is the dimension of the last hidden state of the encoder, `n_out` the size of the output. Dropout of `output_p` is applied. If a `tie_encoder` is passed, it will be used for the weights of the linear layer, that will have `bias` or not.

In [ ]:
show_doc(PoolingLinearClassifier, title_level=3)

<h3 id="PoolingLinearClassifier" class="doc_header"><code>class</code> <code>PoolingLinearClassifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L232" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PoolingLinearClassifier-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PoolingLinearClassifier</code>(**`layers`**:`Collection`\[`int`\], **`drops`**:`Collection`\[`float`\]) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PoolingLinearClassifier-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PoolingLinearClassifier-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PoolingLinearClassifier</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a linear classifier with pooling.  

The last output, `MaxPooling` of all the outputs and `AvgPooling` of all the outputs are concatenated, then blocks of [`bn_drop_lin`](/layers.html#bn_drop_lin) are stacked, according to the values in [`layers`](/layers.html#layers) and `drops`.

## Basic NLP modules

On top of the pytorch or the fastai [`layers`](/layers.html#layers), the language models use some custom layers specific to NLP.

In [ ]:
show_doc(EmbeddingDropout, title_level=3)

<h3 id="EmbeddingDropout" class="doc_header"><code>class</code> <code>EmbeddingDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L57" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#EmbeddingDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>EmbeddingDropout</code>(**`emb`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`embed_p`**:`float`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="EmbeddingDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#EmbeddingDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>EmbeddingDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Apply dropout with probabily `embed_p` to an embedding layer `emb`.  

Each row of the embedding matrix has a probability `embed_p` of being replaced by zeros while the others are rescaled accordingly. 

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[-0.7379, -1.3970, -0.4075, -0.1676,  2.0396,  3.2226,  0.7128],
        [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
        [-3.2579,  2.2972, -1.8704, -0.4090,  2.6477, -1.5015,  0.7158],
        [ 2.1455,  1.0571, -0.6086,  3.5700,  2.6271, -3.1353,  0.7277],
        [-3.7003, -1.8846,  0.2029, -0.6839,  0.2968, -2.0199,  1.3127],
        [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
        [-0.0051,  2.7428,  3.0068,  0.6242,  1.2747,  0.9262,  0.4070],
        [ 1.9312,  3.0524, -1.2806,  1.5910, -2.1789, -0.1636, -3.4924]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, title_level=3)

<h3 id="RNNDropout" class="doc_header"><code>class</code> <code>RNNDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L17" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>RNNDropout</code>(**`p`**:`float`=***`0.5`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="RNNDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>RNNDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Dropout with probability `p` that is consistent on the seq_len dimension.  

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[-2.1156,  0.9734,  0.2428,  0.9396,  0.4072, -0.8197,  0.3718],
          [ 0.4838,  1.3077, -0.8239, -0.6557,  1.3938,  0.6086, -0.2622],
          [ 0.2372, -0.1627,  0.3117, -0.4811, -1.0841, -0.5207, -0.5131]],
 
         [[-0.6924,  0.4122,  0.2517, -1.0120,  0.6808,  0.8800, -0.7463],
          [-0.9498,  0.7655,  0.7471, -0.2767,  1.2155, -0.1042, -2.1443],
          [-1.2342,  1.9187, -0.8481, -0.4115, -1.3223,  1.4266, -1.4150]],
 
         [[ 0.1539,  0.3142,  0.2158,  1.1411,  0.1316,  0.6158, -1.5078],
          [-1.0177, -0.9230,  0.9994,  0.1140,  0.7432,  0.4353,  0.0096],
          [-0.8231,  1.0086,  1.7685,  0.3304, -0.0896, -1.0513, -1.3017]]]),
 tensor([[[-3.0223,  1.3905,  0.0000,  0.0000,  0.5818, -0.0000,  0.5312],
          [ 0.6911,  1.8681, -0.0000, -0.0000,  1.9911,  0.0000, -0.3745],
          [ 0.3389, -0.2324,  0.0000, -0.0000, -1.5487, -0.0000, -0.7331]],
 
         [[-0.9892,  0.5889,  0.3596, -1.4458,  0.9725,  1.2571, -0.0000],
          [-1

In [ ]:
show_doc(WeightDropout, title_level=3)

<h3 id="WeightDropout" class="doc_header"><code>class</code> <code>WeightDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L27" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>WeightDropout</code>(**`module`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`weight_p`**:`float`, **`layer_names`**:`StrList`=***`['weight_hh_l0']`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="WeightDropout-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>WeightDropout</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A module that warps another layer in which some weights will be replaced by 0 during training.  

Applies dropout of probability `weight_p` to the layers in `layer_names` of `module` in training mode. A copy of those weights is kept so that the dropout mask can change at every batch.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[-0.0702,  0.5725],
        [-0.3910,  0.6512],
        [-0.2203, -0.4315],
        [ 0.2750, -0.2917],
        [-0.4890, -0.3094],
        [ 0.4638, -0.3807],
        [-0.2290, -0.6964],
        [ 0.1224,  0.4043]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[-0.0000,  0.0000],
        [-0.6517,  0.0000],
        [-0.0000, -0.7191],
        [ 0.4583, -0.0000],
        [-0.0000, -0.0000],
        [ 0.7730, -0.6345],
        [-0.0000, -1.1607],
        [ 0.2040,  0.6739]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(PositionalEncoding, title_level=3)

<h3 id="PositionalEncoding" class="doc_header"><code>class</code> <code>PositionalEncoding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L11" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PositionalEncoding-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>PositionalEncoding</code>(**`d`**:`int`) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="PositionalEncoding-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PositionalEncoding-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>PositionalEncoding</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Encode the position with a sinusoid.  

In [ ]:
show_doc(DecoderLayer, title_level=3)

<h3 id="DecoderLayer" class="doc_header"><code>class</code> <code>DecoderLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L138" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DecoderLayer-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>DecoderLayer</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="DecoderLayer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DecoderLayer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>DecoderLayer</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Basic block of a Transformer model.  

In [ ]:
show_doc(MultiHeadAttention, title_level=3)

<h3 id="MultiHeadAttention" class="doc_header"><code>class</code> <code>MultiHeadAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L33" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiHeadAttention-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MultiHeadAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`=***`None`***, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="MultiHeadAttention-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiHeadAttention-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MultiHeadAttention</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

MutiHeadAttention.  

In [ ]:
show_doc(MultiHeadRelativeAttention, title_level=3)

<h3 id="MultiHeadRelativeAttention" class="doc_header"><code>class</code> <code>MultiHeadRelativeAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiHeadRelativeAttention-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>MultiHeadRelativeAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`MultiHeadAttention`](/text.models.transformer.html#MultiHeadAttention)

<div class="collapse" id="MultiHeadRelativeAttention-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiHeadRelativeAttention-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MultiHeadRelativeAttention</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

MutiHeadAttention with relative positional encoding.  

In [ ]:
show_doc(SequentialRNN, title_level=3)

<h3 id="SequentialRNN" class="doc_header"><code>class</code> <code>SequentialRNN</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L155" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialRNN-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>SequentialRNN</code>(**\*`args`**) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

<div class="collapse" id="SequentialRNN-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialRNN-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>SequentialRNN</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A sequential module that passes the reset call to its children.  

In [ ]:
show_doc(SequentialRNN.reset)

<h4 id="SequentialRNN.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L157" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#SequentialRNN-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="SequentialRNN-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#SequentialRNN-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Call the `reset` function of [`self.children`](/torch_core.html#children) (if they have one).

In [ ]:
show_doc(dropout_mask)

<h4 id="dropout_mask" class="doc_header"><code>dropout_mask</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L13" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#dropout_mask-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>dropout_mask</code>(**`x`**:`Tensor`, **`sz`**:`Collection`\[`int`\], **`p`**:`float`)

<div class="collapse" id="dropout_mask-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#dropout_mask-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>dropout_mask</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return a dropout mask of the same type as `x`, size `sz`, with probability `p` to cancel an element.  

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286, 0.0000],
        [1.4286, 1.4286, 1.4286, 0.0000, 1.4286, 0.0000, 0.0000],
        [1.4286, 0.0000, 1.4286, 0.0000, 0.0000, 0.0000, 1.4286]])

Such a mask is then expanded in the sequence length dimension and multiplied by the input to do an [`RNNDropout`](/text.models.awd_lstm.html#RNNDropout).

In [ ]:
show_doc(feed_forward)

<h4 id="feed_forward" class="doc_header"><code>feed_forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#feed_forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>feed_forward</code>(**`d_model`**:`int`, **`d_ff`**:`int`, **`ff_p`**:`float`=***`0.0`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***)

<div class="collapse" id="feed_forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#feed_forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>feed_forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(WeightDropout.forward)

<h4 id="WeightDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L44" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**\*`args`**:`ArgStar`)

<div class="collapse" id="WeightDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(EmbeddingDropout.forward)

<h4 id="EmbeddingDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L65" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#EmbeddingDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`words`**:`LongTensor`, **`scale`**:`Optional`\[`float`\]=***`None`***) → `Tensor`

<div class="collapse" id="EmbeddingDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#EmbeddingDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(RNNDropout.forward)

<h4 id="RNNDropout.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L22" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNDropout-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`x`**:`Tensor`) → `Tensor`

<div class="collapse" id="RNNDropout-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNDropout-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(WeightDropout.reset)

<h4 id="WeightDropout.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L51" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#WeightDropout-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="WeightDropout-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#WeightDropout-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(PoolingLinearClassifier.forward)

<h4 id="PoolingLinearClassifier.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L242" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#PoolingLinearClassifier-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

<div class="collapse" id="PoolingLinearClassifier-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#PoolingLinearClassifier-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(LinearDecoder.forward)

<h4 id="LinearDecoder.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L149" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LinearDecoder-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

<div class="collapse" id="LinearDecoder-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LinearDecoder-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section